In [1]:
import lib.xmlreader as xml
import re
import pandas as pd
import os
import gensim
import sys
import numpy as np
import pickle
from lib.utils import tokenize
from collections import Counter

DATA_PATH = '../database/TASS/'

In [129]:
train_docs = xml.readXML(DATA_PATH + "TASS2018/task1-Training.xml")
test_docs  = xml.readXML(DATA_PATH + "TASS2018/task1-Development.xml" )

In [130]:
trn_text   = [t.content  for t in train_docs]
trn_label  = [t.polarity for t in train_docs]

test_text   = [t.content  for t in test_docs]
test_label  = [t.polarity for t in test_docs]

In [146]:
len(trn_text), len(test_text)

(473, 201)

In [131]:
col_names = ['labels','text']

In [132]:
df_train = pd.DataFrame({'text':trn_text, 'labels':trn_label}, columns=col_names)
df_test  = pd.DataFrame({'text':test_text, 'labels':test_label}, columns=col_names)

In [133]:
#CSV_PATH = DATA_PATH + 'csv_data/'

#if not os.path.exists(CSV_PATH):
#    os.makedirs(CSV_PATH)

In [134]:
#df_train.to_csv(DATA_PATH + 'csv_data/train.csv', index=False)
#df_test.to_csv(DATA_PATH  + 'csv_data/test.csv',  index=False)

In [135]:
#df_train = pd.read_csv(DATA_PATH + 'csv_data/train.csv')
#df_test = pd.read_csv(DATA_PATH + 'csv_data/test.csv')

In [136]:
df_train.head()

,labels,text
0,1,"Sin ser fan de Juan Gabriel, siempre supe que ..."
1,0,"Hoy me sentí como grace de ""al fondo hay sitio..."
2,1,hoy me toco visitar a mamá en su descanso eter...
3,1,Las remuneraciones económicas son lo que todos...
4,1,Lo que mas amo de mi escritorio es que hay una...


In [137]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 2 columns):
labels    473 non-null int64
text      473 non-null object
dtypes: int64(1), object(1)
memory usage: 7.5+ KB


In [138]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
labels    201 non-null int64
text      201 non-null object
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [139]:
tokenize(df_train['text'][0], None)

{'original': 'Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.',
 'words': ['sin',
  'ser',
  'fan',
  'de',
  'juan',
  'gabriel',
  'siempre',
  'supe',
  'que',
  'era',
  'una',
  'fuerza',
  'de',
  'la',
  'naturaleza',
  'hoy',
  'escuche',
  'querida',
  'y',
  'me',
  'dio',
  'una',
  'ternura',
  'enorme'],
 'ratio': 0.05,
 'clean': 'sin ser fan de juan gabriel  siempre supe que era una fuerza de la naturaleza  hoy escuche querida   y me dio una ternura enorme',
 'class': None}

In [140]:
len(df_train['text'])

473

In [141]:
LOF_train = []
LOF_test  = []

for i in range(len(df_train['text'])):
    LOF_train.append(tokenize(df_train['text'][i], None)['words'])
    
for i in range(len(df_test['text'])):
    LOF_test.append(tokenize(df_test['text'][i], None)['words'])

In [142]:
print("Size of List Of Words (Train) : ", len(LOF_train))
print("Size of List Of Words (Test)  : ", len(LOF_test))

Size of List Of Words (Train) :  473
Size of List Of Words (Test)  :  201


In [143]:
#LOF_test[0]

In [144]:
cnt = Counter(word for doc in LOF_train + LOF_test for word in doc )

In [145]:
cnt['felicidad'], len(cnt)

(3, 3067)

In [123]:
cnt.most_common()[:10]

[('que', 637),
 ('de', 572),
 ('y', 430),
 ('no', 397),
 ('a', 390),
 ('me', 339),
 ('la', 338),
 ('en', 303),
 ('el', 300),
 ('es', 277)]

In [124]:
import collections

max_vocab = len(cnt)
min_freq = 3
#max_vocab = len(cnt)
#max_vocab = 4000
#min_freq  = 0


itos = [o for o,c in cnt.most_common(max_vocab) if c >= min_freq]

itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

717

In [125]:
len(stoi)

717

In [126]:
train_lm = np.array([[stoi[o] for o in p] for p in LOF_train])
test_lm  = np.array([[stoi[o] for o in p] for p in LOF_test])

In [127]:
SAVE_PATH = '../database/ulmfit/'
np.save(SAVE_PATH + 'tmp/train_labels_2017.npy',trn_label)
np.save(SAVE_PATH + 'tmp/test_labels_2017.npy',test_label)

In [128]:
np.save(SAVE_PATH + 'tmp/train_lm_2017.npy',train_lm)
np.save(SAVE_PATH + 'tmp/test_lm_2017.npy',test_lm)
pickle.dump(itos, open(SAVE_PATH + 'tmp/itos_2017_freq_'+str(min_freq)+'.pkl', 'wb'))